In [1]:
import pickle
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
from SALib.analyze import fast
import sqlite3

In [2]:
with open(os.path.join('yaml', 'params2.yaml'), 'r', encoding='utf-8') as f:
    problem_yaml = f.read()
problem = yaml.safe_load(problem_yaml)

In [3]:
with open("../fastsample1.sam", "r", encoding="utf-8") as f:
    problem_name = list()
    tmp_list, res_list = list(), list()
    count = 0
    f.seek(0)
    flag = 0
    for num, line in enumerate(f):
        if num == 1:
            total = int(line)
            continue
        elif num == 2:
            num_var = int(line)
            continue

        elif num > 2:

            try:
                line = line.rstrip()
                tmp_list = line.split('\t')
                # print(len(tmp_list))
                if len(tmp_list) == num_var and count < total:
                    res_list.append(tmp_list)
                    count += 1
                else:
                    temp_str = f"{num_var} Distributions"
                    if temp_str in line:
                        start1 = num + 2
                        flag = 1
                        count1 = 0
                    
                    if flag and count1 < num_var and num == start1 + count1*6:
                        problem_name.append(line)
                        count1 += 1

            except ValueError:
                pass


paramsets = np.array(res_list, dtype=np.float64)

In [4]:
problem_dict1 = dict()
for each in problem_name:
    for name, value in zip(problem["names"], problem["bounds"]):
        if each == name:
            problem_dict1[each] = value

In [5]:
bounds_list = list()

In [6]:
for each in problem_name:
    bounds_list.append(problem_dict1[each])

In [7]:
problem["bounds"] = bounds_list
problem["names"] = problem_name
problem["num_vars"] = len(bounds_list)

In [8]:
def gen_res_dict(input_dict: dict):
    tmp_df = None
    for key_ in input_dict.keys():
        
        if key_.isnumeric():
            if key_ == "0":
                name = input_dict["names1"]
                tmp_df = pd.DataFrame(input_dict[key_], columns=input_dict["names1"])
            else:
                name = input_dict["names2"]
                tmp_df = pd.DataFrame(input_dict[key_], columns=input_dict["names2"])
            
            yield (key_, name, tmp_df)

In [9]:
def create_sql(path_sql):
    conn = sqlite3.connect(path_sql)
    c = conn.cursor()
    for var_name in col2[: -1]:
        for ni in n_amout:
            table_name1 = "PPN%s%s" % (ni, var_name)
            table_name2 = "WLPN%s%s" % (ni, var_name)
            c.execute(f"""CREATE TABLE {table_name1}
            (ID INTEGER PRIMARY KEY AUTOINCREMENT,
            YEAR INT NOT NULL,
            Parameter CHAR(20) NOT NULL,
            First_order REAL,
            Total_order REAL,
            DOY INT NOT NULL);""")
            c.execute(f"""CREATE TABLE {table_name2}
            (ID INTEGER PRIMARY KEY AUTOINCREMENT,
            YEAR INT NOT NULL,
            Parameter CHAR(20) NOT NULL,
            First_order REAL,
            Total_order REAL,
            DOY INT NOT NULL);""")
            conn.commit()
    conn.close()


In [10]:
n_amout = [0, 90, 180, 270]
col1 = ["LAIMAX", "TAGP", "TWSO", "id"]
col2 = ["LAI", "TAGP", "id"]

In [11]:
from tqdm import tqdm

In [12]:
root_dir = "./pickleFile/FinalRes/"
sql_path = "./time_LAI36.db"
create_sql(sql_path)
conn = sqlite3.connect(sql_path)
for dir_root, _, file_list in os.walk(root_dir):
    for file_name in tqdm(file_list):
        save_name = file_name.split(".")[0]
        prefix_ = save_name[: -4]
        year = save_name[-4:]
        with open(os.path.join(dir_root, file_name), 'rb') as f:
            result_pp = pickle.load(f)
        
        gen_dict = gen_res_dict(result_pp)
        
        for num_, names_l, df1 in gen_dict:
            if num_ == "0":
                continue
            for each in names_l[: -1]:
                # continue
                Y = df1[each]
                Y = Y.to_numpy()
                Si = fast.analyze(problem, Y, print_to_console=False)
                df_sensitivity = pd.DataFrame({
                    "Parameter": problem["names"],
                    "First_order": Si["S1"],
                    "Total_order": Si["ST"]}).set_index("Parameter")
                df_sensitivity["YEAR"] = year
                df_sensitivity["DOY"] = int(num_)
                save_name1 = f"{prefix_}{each}"
                df_sensitivity.to_sql(save_name1, con=conn, chunksize=10000, if_exists='append')

conn.close()

100%|██████████| 16/16 [16:14<00:00, 60.91s/it]


In [89]:
conn.close()

'PPN0'

In [23]:
def df_mean(df1, df2):
    return (df1 + df2) / 2.0

In [26]:

root_dir = "./modelOut/Si/"

In [34]:
root_dir = "./modelOut/Si/"
for var_name in col1[: -1]:
    for ni in n_amout:
        file_name1 = f"WLPN{ni}2021_{var_name}_0.csv"
        file_name2 = f"WLPN{ni}2022_{var_name}_0.csv"
        df1 = pd.read_csv(os.path.join(root_dir, file_name1), header=0, index_col=0)
        df2 = pd.read_csv(os.path.join(root_dir, file_name2), header=0, index_col=0)
        df = df_mean(df1, df2)
        df.to_csv(os.path.join(root_dir, f"WLPN{ni}_{var_name}.csv"))
    

In [88]:
conn.close()

In [76]:
conn = sqlite3.connect('time_series.db')

In [74]:
c = conn.cursor()

In [75]:
conn = sqlite3.connect('time_series.db')
c = conn.cursor()
for var_name in col2[: -1]:
    for ni in n_amout:
        table_name1 = "PPN%s%s" % (ni, var_name)
        # table_name2 = "PPN%s2022%s" % (ni, var_name)
        c.execute(f"""CREATE TABLE {table_name1}
        (ID INTEGER PRIMARY KEY AUTOINCREMENT,
        YEAR INT NOT NULL,
        Parameter CHAR(20) NOT NULL,
        First_order REAL,
        Total_order REAL,
        DOY INT NOT NULL);""")
        conn.commit()
conn.close()

In [ ]:
df1["id"] = 1

In [ ]:
df1

,First_order,Total_order,id
Parameter,,,
SLATB1,0.096058,0.191488,1
SLATB2,0.008117,0.016103,1
SLATB3,0.000034,0.003235,1
KDIFTB0,0.035883,0.056186,1
KDIFTB2,0.000685,0.013220,1
EFFTB1,0.008553,0.015379,1
EFFTB2,0.091203,0.158128,1
AMAXTB125,0.007670,0.015771,1
AMAXTB150,0.000612,0.001724,1
